In [1]:
#imports
import numpy as np
import sklearn
from skimage import morphology
import cv2
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.transform import rotate
from sklearn.cluster import KMeans

imgDir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, "data"))

# B-Feature: Irregular Border


In [2]:
# def find_midpoint_v1(image):
    
#     row_mid = image.shape[0] / 2     # nr of the middle row
#     col_mid = image.shape[1] / 2     # nr of the middle column
#     return row_mid, col_mid

def find_midpoint_v4(mask):
        """Finds horizontal and vertical midpoints of the given binary
        mask, such that 50% of nonzero pixels in the mask
        are on either side of midpoint.

        :param mask: Binary Mask to be examined.
        :return x, y: coordinates of midpoint."""
        mX = 0
        mY = 0
        #get horizontal vector which contains number of nonzero pixels for each mask column
        summedX = np.sum(mask, axis=0)
        #calculate 50% of nonzero pixels in mask as threshold
        half_sumX = np.sum(summedX) / 2
        #iterate through columns until half of nonzero pixels in mask have been reached
        for i, n in enumerate(np.add.accumulate(summedX)):
            if n > half_sumX:
                #x-coordinate at which 50% of nonzero pixels where exceeded
                mX = i
                break
        
        summedY = np.sum(mask, axis=1)
        #calculate 50% of nonzero pixels in mask as threshold
        half_sumY = np.sum(summedY) / 2
        #iterate through rows until half of nonzero pixels in mask have been reached
        for i, n in enumerate(np.add.accumulate(summedY)):
            if n > half_sumY:
                # y-coordinate at which 50% of nonzero pixels where exceeded
                mY = i
                break

        return mX, mY
        

def cut_mask(mask):
    
    # input is numpy array mask
    col_sums = np.sum(mask, axis=0)     # sums up the values between 0 and 1
    row_sums = np.sum(mask, axis=1)     # shows if any row or column contains anything but 0s

    active_cols = []        # lists all the columns where there is no lesion
    for index, col_sum in enumerate(col_sums):  # takes all columns
        if col_sum != 0:                        # if the full column is 0, it's not added to the list
            active_cols.append(index)

    active_rows = []        # analog for rows
    for index, row_sum in enumerate(row_sums):
        if row_sum != 0:
            active_rows.append(index)

    # taking the bordering rows and columns of the mask (excluding the black edges where there is nothing)
    col_min = active_cols[0]
    col_max = active_cols[-1]
    row_min = active_rows[0]
    row_max = active_rows[-1]

    # saving the new mask
    cut_mask_ = mask[row_min:row_max+1, col_min:col_max+1]

    return cut_mask_

def cut_im_by_mask(image, mask):
    
    # same as previous function
    col_sums = np.sum(mask, axis=0)
    row_sums = np.sum(mask, axis=1)

    active_cols = []
    for index, col_sum in enumerate(col_sums):
        if col_sum != 0:
            active_cols.append(index)

    active_rows = []
    for index, row_sum in enumerate(row_sums):
        if row_sum != 0:
            active_rows.append(index)

    col_min = active_cols[0]
    col_max = active_cols[-1]
    row_min = active_rows[0]
    row_max = active_rows[-1]

    #except the cutting is applied to the image itself and not the mask
    cut_image = image[row_min:row_max+1, col_min:col_max+1]

    return cut_image

def getSectorMask(mask, angleStart, angleEnd, mX, mY):
    # Get image dimensions
    height, width = mask.shape[:2]

    # Create coordinate grid
    Y, X = np.ogrid[:height, :width]
    dx = X - mX
    dy = mY - Y  # inverted Y to make 0° at the top (like a clock)

    # Compute angle in degrees (0° at top, increasing clockwise)
    angles = (np.degrees(np.arctan2(dx, dy)) + 360) % 360

    # Create mask for angles between angleStart and angleEnd
    mask = (angles >= angleStart) & (angles < angleEnd)

    return mask

def getMeanGradient(img, mask):
    """For the given image, return the mean
    gradient magnitude of the top 10% of gradients
    in the masked region."""

    imgGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    sobelX = cv2.Sobel(imgGray, cv2.CV_64F, 1, 0, ksize=3) #get horizontal gradient for whole image
    sobelY = cv2.Sobel(imgGray, cv2.CV_64F, 0, 1, ksize=3) #get vertical gradient for whole image
    gradMag = np.sqrt(sobelX**2 + sobelY**2)

    #compute top 10% of gradientMagnitudes:
    gradMagFlat = gradMag[mask].flatten()#flatten array and take only masked values

    n10 = int(len(gradMagFlat)*0.1)#obtain the size of 10th percentile
    gradMagSort = sorted(gradMagFlat, reverse=True)#sort descending

    return np.mean(gradMagSort[:n10])#return mean gradient magnitude over top10% of gradient magnitudes in masked area

def irregularBorder(img, mask, nSectors=12) -> int:
    """Extract the \"irregular Boder\" feature,
    which is a number from 0 to 1 that is a measure
    for the difference between the center intensity
    and border intensity of the lesion.
    
    :param img: the image to process
    :param mask: mask to apply to the image
    :return: border irregularity measure from
    0(regular) to 1(irregular)"""
    cutImg = cut_im_by_mask(img, mask)
    cutMask = cut_mask(mask)
    mX, mY = find_midpoint_v4(cutMask)

    #store distanceScores
    gradientScores = []

    testMask = np.zeros(cutMask.shape, dtype=np.uint8)#mask for displaying and testing


    sectorDeg = int(np.ceil(360 / nSectors))
    for deg in range(0, 360, sectorDeg):
        sectorMask = (getSectorMask(cutMask, deg, deg+sectorDeg, mX, mY) & cutMask)#extract current sector (only where lesion is)
        #extract only border pixels
        y_nonzero, x_nonzero = np.nonzero(sectorMask)#get vectors of x-values & y-values of nonzero points (x&y reversed by numpy)
        dX = x_nonzero - mX#get vector of x-distances from points to middle
        dY = y_nonzero - mY#get vector of y-distances from points to middle
        dist = np.sqrt(dX**2 + dY**2)#compute distances for all of these points
        maxDist = np.max(dist)#get maximum distance
        distFar = maxDist*0.8#set inner threshold for border pixels
        idcBorder = np.where(dist > distFar)#extract indices for points that exceed the distance threshold
        borderMask = np.zeros(cutMask.shape, dtype=np.uint8)#create mask for border pixels
        borderMask[y_nonzero[idcBorder], x_nonzero[idcBorder]] = 1#use index obtained earlier to obtain coordinates of points exceeding the distance (again, x&y reversed because of numpy)
        #append mean gradient for border in this sector
        gradientScores.append(getMeanGradient(img, borderMask))

        testMask[y_nonzero[idcBorder], x_nonzero[idcBorder]] = 1#test mask

    plt.imshow(cutImg)
    #plt.imshow(testMask, cmap="Reds", alpha=0.3)
    plt.scatter(mX, mY, c="red", marker="x", s=100)
    plt.axis("off")
    plt.show()

    return np.mean(gradientScores)


#imName = "PAT_72_110_647.png"
#imName = "PAT_20_30_44.png"
#imName = "PAT_83_126_820.png"
imName = "PAT_90_138_369.png"#good test case to show where it goes wrong -> if the mask doesn't fit properly, then it's fucked -> too high
#imName = "PAT_173_268_1000.png"#another example of a not-so spot on mask -> too high
#imName = "PAT_187_287_482.png"#kinda okay
#imName = "PAT_320_681_410.png"#kinda okay
#imName= "PAT_333_702_840.png"
#imName = "PAT_364_748_278.png"
maskName = imName.split(".")[0] + "_mask" + ".png"
testImg = cv2.imread(os.path.join(imgDir, imName))
testImg = cv2.cvtColor(testImg, cv2.COLOR_BGR2RGB)
testMask = cv2.imread(os.path.join(imgDir, maskName), cv2.IMREAD_GRAYSCALE)
_, testMask = cv2.threshold(testMask, 127, 255, cv2.THRESH_BINARY)

print(f"Irregular Border: {irregularBorder(testImg, testMask)}")

KeyboardInterrupt: 